In [1]:
# Common packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific packages
import random # For functions sample() and shuffle()
# To rescale numeric variables
from sklearn.preprocessing import MinMaxScaler
# To one-hot encode cat. variables
from sklearn.preprocessing import OneHotEncoder

In [2]:

##### Loading the data #####
hist_data_df = pd.read_csv('chap9-historical_data.csv')
exp_data_df = pd.read_csv('chap9-experimental_data.csv')

### Minor data formatting

# Reformating categorical and id variables
hist_data_df['tier'] = pd.Categorical(hist_data_df.tier, categories=[3,2,1], ordered = True)
hist_data_df['ID'] = hist_data_df.ID.astype(str)
exp_data_df['tier'] = pd.Categorical(exp_data_df.tier, categories=[3,2,1], ordered = True)
exp_data_df['ID'] = exp_data_df.ID.astype(str)


In [3]:

def no_strat_assgnt(df, Nexp, k):
    temp = pd.DataFrame({'ID': df.ID.unique()})
    temp = temp.sample(Nexp)
    grp = list(range(k)) * int(Nexp / k)
    random.shuffle(grp)
    temp['grp'] = grp
    return temp

no_strat_assgnt(hist_data_df, 2000, k=4)

,ID,grp
1834,1835,3
760,761,2
200,201,1
2640,2641,1
4959,4960,1
...,...,...
5,6,2
74,75,2
4431,4432,1
1178,1179,2


In [4]:
hist_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ID          175000 non-null  object  
 1   period      175000 non-null  int64   
 2   month       175000 non-null  int64   
 3   sq_ft       175000 non-null  float64 
 4   tier        175000 non-null  category
 5   avg_review  175000 non-null  float64 
 6   BPday       175000 non-null  float64 
dtypes: category(1), float64(3), int64(2), object(1)
memory usage: 8.2+ MB


In [5]:
def strat_prep_fun(df):
    temp = df.copy()

    temp = temp.groupby(['ID','tier'], observed=False).agg(
        sq_ft = ('sq_ft','mean'),
        avg_review = ('avg_review','mean'),
        BPday = ('BPday','mean')
    )
    temp = temp.dropna().reset_index()

    num_df = temp.copy().select_dtypes('float64')
    cat_df = temp.copy().select_dtypes('category')

    scaler = MinMaxScaler()
    scaler.fit(num_df)
    num_np = scaler.transform(num_df)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(cat_df)
    cat_np = enc.transform(cat_df).toarray()

    data_np = np.concatenate((num_df, cat_np), axis=1)
    del num_df, num_np, cat_df, enc, scaler
    return data_np

prepared = strat_prep_fun(hist_data_df)

prepared

array([[821.67548629,   9.39342726,  45.93411704,   0.        ,
          1.        ,   0.        ],
       [977.68632137,  10.        ,  47.02031206,   0.        ,
          1.        ,   0.        ],
       [772.24643725,   5.05391337,  36.03493243,   1.        ,
          0.        ,   0.        ],
       ...,
       [931.23973925,   7.82448952,  42.00859311,   0.        ,
          0.        ,   1.        ],
       [792.20446113,   4.60368051,  32.83121812,   0.        ,
          0.        ,   1.        ],
       [859.06503328,   7.91434225,  42.09540755,   0.        ,
          0.        ,   1.        ]])

In [6]:
def stratified_assgnt_fun(dat_df, K):
    
    #Sampling down to a multiple of our number of groups
    remainder = len(dat_df) % K
    if remainder != 0:
        dat_df = dat_df.sample(len(dat_df) - remainder)
      
    dat_ID = dat_df.ID.astype(str).tolist() # Extract ID for later join

    match_len = K - 1 # Number of matches we want to find
    match_idx = match_len - 1 # Accounting for 0-indexing
    
    data_np = strat_prep_fun(dat_df)
    N = len(data_np)
    
    #Calculate distance matrix
    from scipy.spatial import distance_matrix
    d_mat = distance_matrix(data_np, data_np)
    np.fill_diagonal(d_mat,N+1)
    # Set up variables
    available = [i for i in range(N)]
    available_temp = available.copy()
    matches_lst = []
    lim = int(N/match_len)
    
    closest = np.argpartition(d_mat, kth=match_idx,axis=1)
    
    for n in available:
        #print("n = ", n)
        if len(matches_lst) == lim: break
        if n in available_temp:
            for match_lim in range(match_idx,N-1):
                #print("match_lim = ", match_lim)
                possible_matches = closest[n,:match_lim].tolist()
                matches = list(set(available_temp) & set(possible_matches))
                #print("len(matches) = ",  len(matches))
                if len(matches) == match_len:
                    matches.append(n)
                    matches_lst.append(matches)
                    available_temp = [m for m in available_temp if m not in matches]
                    break
                else:
                    closest[n,:] = np.argpartition(d_mat[n,:], kth=match_lim)
                    
    #Assigning experimental groups to the matched sets
    exp_grps = np.array(list(range(K))*(int(N/K))).reshape((int(N/K),K))
    exp_grps = exp_grps.tolist()
    for j in exp_grps: 
        np.random.shuffle(j)
    #flattening the two lists
    import itertools
    exp_grps = list(itertools.chain(*exp_grps))
    matches_lst2 = list(itertools.chain(*matches_lst))
    exp_grps2 = [x for _,x in sorted(zip(matches_lst2,exp_grps))]
    
    assgnt_df = pd.DataFrame(exp_grps2, columns=['grp'])
    assgnt_df.grp = assgnt_df.grp.astype(str)
    assgnt_df.grp.loc[assgnt_df.grp == '0'] = 'ctrl'
    assgnt_df.grp.loc[assgnt_df.grp == '1'] = 'treat1'
    assgnt_df.grp.loc[assgnt_df.grp == '2'] = 'treat2'
    
    
    assgnt_df['ID'] = dat_ID
    dat_df = dat_df.merge(assgnt_df, on='ID', how='inner')
    return dat_df

In [7]:
# metric function for minimum-duration
def treat2_metric_fun(df):
    model = ols("BPday~sq_ft+tier+avg_review+grp", data=df)
    res = model.fit(disp=0)
    coeff = res.params['grp[T.treat2]']
    return coeff

# determining the bootstrap sample CI 
def boot_CI_fun(df, metric_fun, B=100, conf_level = 0.9):
    # set sample size
    N = len(df)
    coeffs = []

    for i in range(B):
        sim_df =  df.sample(n=N, replace=True)
        coeff = metric_fun(sim_df)
        coeffs.append(coeff)
    
    coeffs.sort()
    start_idx = round(B * (1 -conf_level) / 2)
    end_idx = - round(B * (1 - conf_level) / 2)
    confint = [coeffs[start_idx], coeffs[end_idx]]
    return confint

# decision function
def decision_fun(df, metric_fun, B = 100, conf_level = 0.9):
    boot_CI = boot_CI_fun(df, metric_fun, B = B, conf_level=conf_level)
    decision = 1 if boot_CI[0] > 0 else 0
    return decision

def single_sim_fun(df, metric_fun, Nexp, eff_size, B = 100, conf_level = 0.9):

    # filter data to a ranodm month
    per = random.sample(range(35), 1)[0] + 1
    df = df.loc[df.period == per]
    df = df.sample(n=Nexp)

    # prepare stratified assignemnt for a random sample of desired size
    sample_df = df.sample(Nexp)
    sim_df = stratified_assgnt_fun(sample_df, K = 3)

    # add target effect size
    sim_df.BPday = np.where(
        sim_df.grp == 'treat2',
        sim_df.BPday + eff_size, 
        sim_df.BPday
    )

    # calculate decision (should be 1 for true effect)
    decision = decision_fun(
        sim_df,
        metric_fun, 
        B = B,
        conf_level=conf_level
        )

    return decision

def power_sim_fun(df, metric_fun, Nexp, eff_size, Nsim, B = 100, conf_level = 0.9):
    power_list = []
    for i in range(Nsim):
        power_list.append(
            single_sim_fun(
                df,
                metric_fun=metric_fun,
                Nexp=Nexp, 
                eff_size = eff_size,
                B = B, 
                conf_level= conf_level
                )
            )
    power = np.mean(power_list)
    return power
    

In [8]:
power = power_sim_fun(hist_data_df, treat2_metric_fun, Nexp = 1500, eff_size = 2, 
                      Nsim = 100, B = 100, conf_level = 0.9)

power

0.74

In [9]:
exp_data_reg_df = exp_data_df.copy()
exp_data_reg_df.BPday = np.where(
    (exp_data_reg_df.compliant == 1) & (exp_data_reg_df.grp == 'treat2'),
    exp_data_reg_df.BPday -10,
    exp_data_reg_df.BPday
)

ols("BPday~sq_ft+tier+avg_review+grp", data=exp_data_reg_df).fit(disp=0).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  BPday   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     12.26
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           1.68e-13
Time:                        10:52:39   Log-Likelihood:                -6087.8
No. Observations:                1500   AIC:                         1.219e+04
Df Residuals:                    1493   BIC:                         1.223e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        19.2328      3.574      5.382      0.000      12.223      26.242
tier[T.2]         1.0596      0.841      1.261      0.208      -0.589       2.708
tier[T.1]         5.1705      1.036      4.990      0.000       3.138       7.203
grp[T.treat1]     0.9669      0.889      1.088      0.277      -0.776       2.710
grp[T.treat2]    -0.1726      0.888     -0.194      0.846      -1.915       1.570
sq_ft             0.0068      0.004      1.838      0.066      -0.000       0.014
avg_review        1.6926      0.254      6.675      0.000       1.195       2.190
==============================================================================
Omnibus:                        6.088   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                5.994
Skew:                           0.149   Prob(JB):                       0.0499
Kurtosis:                       3.082   Cond. No.                     7.95e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
exp_data_reg_df.groupby('grp').agg(compliance_rate = ('compliant','mean'))

,compliance_rate
grp,
ctrl,1.000
treat1,0.238
treat2,0.166


In [12]:
grp_1_coeff = 0.97
grp_1_compliance = 0.24

CACE_1 = grp_1_coeff / grp_1_compliance
CACE_1

4.041666666666667